    dis.007

Import libraries

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob
from matplotlib import pyplot
%matplotlib inline

Define local file locations

In [3]:
local_folder = "C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/bio.008/"
file_name = "impact.asc"
file_name2 = "sp_richness.asc"

local_orig = local_folder + file_name
local_orig2 = local_folder + file_name2

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "_edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "_edit.tif" 

s3 tools

In [5]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/bio_008_cumulative_climate_impacts_on_marine_ecosystems/"

s3_file = "bio_008_impact.tif"
s3_file2 = "bio_008_sp_richness.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [6]:
files = [local_orig, local_orig2]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 360, 'height': 180, 'count': 1, 'crs': None, 'transform': Affine(1.0, 0.0, -180.0,
       0.0, -1.0, 90.0), 'tiled': False}
{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 720, 'height': 341, 'count': 1, 'crs': None, 'transform': Affine(0.5, 0.0, -180.0,
       0.0, -0.5, 84.917235592), 'tiled': False}


Use rasterio to reproject and compress

In [7]:
os.getcwd()
os.chdir(local_folder)
os.environ["local_orig"] =local_orig
os.environ["local_edit"] =local_edit
os.environ["local_orig2"] =local_orig2
os.environ["local_edit2"] =local_edit2

In [8]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig% %local_edit%

0...10...20...30...40...50...60...70...80...90...100 - done.


In [9]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig2% %local_edit2%

0...10...20...30...40...50...60...70...80...90...100 - done.


In [10]:
files = [local_orig, local_edit]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 360, 'height': 180, 'count': 1, 'crs': None, 'transform': Affine(1.0, 0.0, -180.0,
       0.0, -1.0, 90.0), 'tiled': False}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 360, 'height': 180, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(1.0, 0.0, -180.0,
       0.0, -1.0, 90.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


Upload orig and edit files to s3

In [18]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))

C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/bio.008/sp_richness_edit.tif  308228 / 308228.0  (100.00%)

In [12]:
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_edit
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_edit
os.environ["Zgs_key2"] = "gs://resource-watch-public/" + s3_key_edit2
os.environ["Zs3_key2"] = "s3://wri-public-data/" + s3_key_edit2

In [8]:
!echo %Zs3_key%
!echo %Zgs_key%
!echo %Zs3_key%
!echo %Zgs_key%

s3://wri-public-data/resourcewatch/raster/dis_007_landslide_susceptibility_map/dis_007_landslide_susceptibility_map_edit.tif
gs://resource-watch-public/resourcewatch/raster/dis_007_landslide_susceptibility_map/dis_007_landslide_susceptibility_map_edit.tif


In [13]:
!gsutil cp %Zs3_key% %Zgs_key%
!gsutil cp %Zs3_key2% %Zgs_key2%

Copying s3://wri-public-data/resourcewatch/raster/bio_008_cumulative_climate_impacts_on_marine_ecosystems/bio_008_impact_edit.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/144.9 KiB]                                                
/ [1 files][144.9 KiB/144.9 KiB]                                                
-

Operation completed over 1 objects/144.9 KiB.                                    
Copying s3://wri-public-data/resourcewatch/raster/bio_008_cumulative_climate_impacts_on_marine_ecosystems/bio_008_sp_richness_edit.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/301.0 KiB]                                                
/ [1 files][301.0 KiB/301.0 KiB]                                                
-

Operation completed over 1 objects/301.0 KiB.                                    


In [ ]:
gsutil cp s3://wri-public-data/resourcewatch/raster/dis_007_landslide_susceptibility_map/dis_007_landslide_susceptibility_map_edit.tif gs://resource-watch-public/resourcewatch/raster/dis_007_landslide_susceptibility_map/dis_007_landslide_susceptibility_map_edit.tif

In [ ]:
# # with rio.open(local_orig) as src:
#     data = src.read(indexes=1)
#     pyplot.imshow(data)

In [ ]:
# with rio.open(local_edit) as src:
#     data = src.read(indexes=1)
#     pyplot.imshow(data)

In [17]:
os.environ["asset_id"] = "users/resourcewatch/bio_008_cumulative_climate_impacts"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%
os.environ["asset_id2"] = "users/resourcewatch/bio_008_species_richness"
!earthengine upload image --asset_id=%asset_id2% %Zgs_key2%

Started upload task with ID: N6FTRJEMZRXVGB44IACUIJDG
Started upload task with ID: OQLVUFSOHFQK5OQGDRPZXFCJ


In [14]:
!earthengine task info 3QFYIBV2XBJCHZK4ZQJQECAR

3QFYIBV2XBJCHZK4ZQJQECAR:
  State: FAILED
  Type: Upload
  Description: Asset ingestion: users/resourcewatch/dis_007_landslide_susceptibility_map
  Created: 2017-10-06 15:19:45.257000
  Started: 2017-10-06 15:19:46.971000
  Updated: 2017-10-06 15:19:47.625000
  Error: Cannot overwrite asset 'users/resourcewatch/dis_007_landslide_susceptibility_map'.
